In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# miscellaneous
import builtins
import functools
# import bisect
# import shutil
import time
import json
# data generation
import dlrm_data_pytorch as dp

# numpy
import numpy as np

# onnx
# The onnx import causes deprecation warnings every time workers
# are spawned during testing. So, we filter out those warnings.
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)
import onnx

# pytorch
import torch
import torch.nn as nn
from torch.nn.parallel.parallel_apply import parallel_apply
from torch.nn.parallel.replicate import replicate
from torch.nn.parallel.scatter_gather import gather, scatter
# quotient-remainder trick
from tricks.qr_embedding_bag import QREmbeddingBag
# mixed-dimension trick
from tricks.md_embedding_bag import PrEmbeddingBag, md_solver

import sklearn.metrics

# from torchviz import make_dot
# import torch.nn.functional as Functional
# from torch.nn.parameter import Parameter

exc = getattr(builtins, "IOError", "FileNotFoundError")

In [2]:
### import packages ###
import sys
import argparse

In [3]:
### define dlrm in PyTorch ###
class DLRM_Net(nn.Module):
    def create_mlp(self, ln, sigmoid_layer):
        # build MLP layer by layer
        layers = nn.ModuleList()
        for i in range(0, ln.size - 1):
            n = ln[i]
            m = ln[i + 1]

            # construct fully connected operator
            LL = nn.Linear(int(n), int(m), bias=True)

            # initialize the weights
            # with torch.no_grad():
            # custom Xavier input, output or two-sided fill
            mean = 0.0  # std_dev = np.sqrt(variance)
            std_dev = np.sqrt(2 / (m + n))  # np.sqrt(1 / m) # np.sqrt(1 / n)
            W = np.random.normal(mean, std_dev, size=(m, n)).astype(np.float32)
            std_dev = np.sqrt(1 / m)  # np.sqrt(2 / (m + 1))
            bt = np.random.normal(mean, std_dev, size=m).astype(np.float32)
            # approach 1
            LL.weight.data = torch.tensor(W, requires_grad=True)
            LL.bias.data = torch.tensor(bt, requires_grad=True)
            # approach 2
            # LL.weight.data.copy_(torch.tensor(W))
            # LL.bias.data.copy_(torch.tensor(bt))
            # approach 3
            # LL.weight = Parameter(torch.tensor(W),requires_grad=True)
            # LL.bias = Parameter(torch.tensor(bt),requires_grad=True)
            layers.append(LL)

            # construct sigmoid or relu operator
            if i == sigmoid_layer:
                layers.append(nn.Sigmoid())
            else:
                layers.append(nn.ReLU())

        # approach 1: use ModuleList
        # return layers
        # approach 2: use Sequential container to wrap all layers
        return torch.nn.Sequential(*layers)

    def create_emb(self, m, ln):
        # litez: m is embedding_dim;
        # ln gives a list of vocabulary size for each 
        # of the sparse feature
        emb_l = nn.ModuleList()
        for i in range(0, ln.size):
            n = ln[i]
            # construct embedding operator
            if self.qr_flag and n > self.qr_threshold:
                EE = QREmbeddingBag(n, m, self.qr_collisions,
                    operation=self.qr_operation, mode="sum", sparse=True)
            elif self.md_flag and n > self.md_threshold:
                _m = m[i]
                base = max(m)
                EE = PrEmbeddingBag(n, _m, base)
                # use np initialization as below for consistency...
                W = np.random.uniform(
                    low=-np.sqrt(1 / n), high=np.sqrt(1 / n), size=(n, _m)
                ).astype(np.float32)
                EE.embs.weight.data = torch.tensor(W, requires_grad=True)

            else:
                EE = nn.EmbeddingBag(n, m, mode="sum", sparse=True)

                # initialize embeddings
                # nn.init.uniform_(EE.weight, a=-np.sqrt(1 / n), b=np.sqrt(1 / n))
                W = np.random.uniform(
                    low=-np.sqrt(1 / n), high=np.sqrt(1 / n), size=(n, m)
                ).astype(np.float32)
                # approach 1
                EE.weight.data = torch.tensor(W, requires_grad=True)
                # approach 2
                # EE.weight.data.copy_(torch.tensor(W))
                # approach 3
                # EE.weight = Parameter(torch.tensor(W),requires_grad=True)

            emb_l.append(EE)

        return emb_l

    def __init__(
        self,
        m_spa=None,
        ln_emb=None,
        ln_bot=None,
        ln_top=None,
        arch_interaction_op=None,
        arch_interaction_itself=False,
        sigmoid_bot=-1,
        sigmoid_top=-1,
        sync_dense_params=True,
        loss_threshold=0.0,
        ndevices=-1,
        qr_flag=False,
        qr_operation="mult",
        qr_collisions=0,
        qr_threshold=200,
        md_flag=False,
        md_threshold=200,
    ):
        super(DLRM_Net, self).__init__()

        if (
            (m_spa is not None)
            and (ln_emb is not None)
            and (ln_bot is not None)
            and (ln_top is not None)
            and (arch_interaction_op is not None)
        ):

            # save arguments
            self.ndevices = ndevices
            self.output_d = 0
            self.parallel_model_batch_size = -1
            self.parallel_model_is_not_prepared = True
            self.arch_interaction_op = arch_interaction_op
            self.arch_interaction_itself = arch_interaction_itself
            self.sync_dense_params = sync_dense_params
            self.loss_threshold = loss_threshold
            # create variables for QR embedding if applicable
            self.qr_flag = qr_flag
            if self.qr_flag:
                self.qr_collisions = qr_collisions
                self.qr_operation = qr_operation
                self.qr_threshold = qr_threshold
            # create variables for MD embedding if applicable
            self.md_flag = md_flag
            if self.md_flag:
                self.md_threshold = md_threshold
            # create operators
            if ndevices <= 1:
                self.emb_l = self.create_emb(m_spa, ln_emb)
            self.bot_l = self.create_mlp(ln_bot, sigmoid_bot)
            self.top_l = self.create_mlp(ln_top, sigmoid_top)

    def apply_mlp(self, x, layers):
        # approach 1: use ModuleList
        # for layer in layers:
        #     x = layer(x)
        # return x
        # approach 2: use Sequential container to wrap all layers
        return layers(x)

    def apply_emb(self, lS_o, lS_i, emb_l):
        # WARNING: notice that we are processing the batch at once. We implicitly
        # assume that the data is laid out such that:
        # 1. each embedding is indexed with a group of sparse indices,
        #   corresponding to a single lookup
        # 2. for each embedding the lookups are further organized into a batch
        # 3. for a list of embedding tables there is a list of batched lookups

        ly = []
        #litez: lS_i is a list of tensors, each of which is of size (Batch_size,1)\
        #Since num_of_sparse elements is 26, we have len(lS_i)=26 in this case.
        for k, sparse_index_group_batch in enumerate(lS_i):
            sparse_offset_group_batch = lS_o[k]

            # embedding lookup
            # We are using EmbeddingBag, which implicitly uses sum operator.
            # The embeddings are represented as tall matrices, with sum
            # happening vertically across 0 axis, resulting in a row vector
            E = emb_l[k]
            # litez: V.shape =(batch_size, embedding size);
            # what is the advantage of using embedding bag?
            V = E(sparse_index_group_batch, sparse_offset_group_batch)

            ly.append(V)

        # print(ly)
        return ly

    def interact_features(self, x, ly):
        if self.arch_interaction_op == "dot":
            # concatenate dense and sparse features
            (batch_size, d) = x.shape
            T = torch.cat([x] + ly, dim=1).view((batch_size, -1, d))
            # perform a dot product
            Z = torch.bmm(T, torch.transpose(T, 1, 2))
            # append dense feature with the interactions (into a row vector)
            # approach 1: all
            # Zflat = Z.view((batch_size, -1))
            # approach 2: unique
            _, ni, nj = Z.shape
            # approach 1: tril_indices
            # offset = 0 if self.arch_interaction_itself else -1
            # li, lj = torch.tril_indices(ni, nj, offset=offset)
            # approach 2: custom
            offset = 1 if self.arch_interaction_itself else 0
            li = torch.tensor([i for i in range(ni) for j in range(i + offset)])
            lj = torch.tensor([j for i in range(nj) for j in range(i + offset)])
            Zflat = Z[:, li, lj]
            # concatenate dense features and interactions
            R = torch.cat([x] + [Zflat], dim=1)
        elif self.arch_interaction_op == "cat":
            # concatenation features (into a row vector)
            R = torch.cat([x] + ly, dim=1)
        else:
            sys.exit(
                "ERROR: --arch-interaction-op="
                + self.arch_interaction_op
                + " is not supported"
            )

        return R

    def forward(self, dense_x, lS_o, lS_i):
        if self.ndevices <= 1:
            return self.sequential_forward(dense_x, lS_o, lS_i)
        else:
            return self.parallel_forward(dense_x, lS_o, lS_i)

    def sequential_forward(self, dense_x, lS_o, lS_i):
        # process dense features (using bottom mlp), resulting in a row vector
        x = self.apply_mlp(dense_x, self.bot_l)
        # debug prints
        # print("intermediate")
        # print(x.detach().cpu().numpy())

        # process sparse features(using embeddings), resulting in a list of row vectors
        # litez: ly is a list of (Batch_size, embedding_size); list lengh equals the number of
        # sparese features
        ly = self.apply_emb(lS_o, lS_i, self.emb_l)
        # for y in ly:
        #     print(y.detach().cpu().numpy())

        # interact features (dense and sparse)
        # litez: z is of shape(batch_size, total_number of features feeding into the top layer mlp)
        z = self.interact_features(x, ly)
        # print(z.detach().cpu().numpy())

        # obtain probability of a click (using top mlp)
        p = self.apply_mlp(z, self.top_l)

        # clamp output if needed
        if 0.0 < self.loss_threshold and self.loss_threshold < 1.0:
            z = torch.clamp(p, min=self.loss_threshold, max=(1.0 - self.loss_threshold))
        else:
            z = p

        return z

    def parallel_forward(self, dense_x, lS_o, lS_i):
        ### prepare model (overwrite) ###
        # WARNING: # of devices must be >= batch size in parallel_forward call
        batch_size = dense_x.size()[0]
        ndevices = min(self.ndevices, batch_size, len(self.emb_l))
        device_ids = range(ndevices)
        # WARNING: must redistribute the model if mini-batch size changes(this is common
        # for last mini-batch, when # of elements in the dataset/batch size is not even
        if self.parallel_model_batch_size != batch_size:
            self.parallel_model_is_not_prepared = True

        if self.parallel_model_is_not_prepared or self.sync_dense_params:
            # replicate mlp (data parallelism)
            self.bot_l_replicas = replicate(self.bot_l, device_ids)
            self.top_l_replicas = replicate(self.top_l, device_ids)
            self.parallel_model_batch_size = batch_size

        if self.parallel_model_is_not_prepared:
            # distribute embeddings (model parallelism)
            t_list = []
            for k, emb in enumerate(self.emb_l):
                d = torch.device("cuda:" + str(k % ndevices))
                emb.to(d)
                t_list.append(emb.to(d))
            self.emb_l = nn.ModuleList(t_list)
            self.parallel_model_is_not_prepared = False

        ### prepare input (overwrite) ###
        # scatter dense features (data parallelism)
        # print(dense_x.device)
        dense_x = scatter(dense_x, device_ids, dim=0)
        # distribute sparse features (model parallelism)
        if (len(self.emb_l) != len(lS_o)) or (len(self.emb_l) != len(lS_i)):
            sys.exit("ERROR: corrupted model input detected in parallel_forward call")

        t_list = []
        i_list = []
        for k, _ in enumerate(self.emb_l):
            d = torch.device("cuda:" + str(k % ndevices))
            t_list.append(lS_o[k].to(d))
            i_list.append(lS_i[k].to(d))
        lS_o = t_list
        lS_i = i_list

        ### compute results in parallel ###
        # bottom mlp
        # WARNING: Note that the self.bot_l is a list of bottom mlp modules
        # that have been replicated across devices, while dense_x is a tuple of dense
        # inputs that has been scattered across devices on the first (batch) dimension.
        # The output is a list of tensors scattered across devices according to the
        # distribution of dense_x.
        x = parallel_apply(self.bot_l_replicas, dense_x, None, device_ids)
        # debug prints
        # print(x)

        # embeddings
        ly = self.apply_emb(lS_o, lS_i, self.emb_l)
        # debug prints
        # print(ly)

        # butterfly shuffle (implemented inefficiently for now)
        # WARNING: Note that at this point we have the result of the embedding lookup
        # for the entire batch on each device. We would like to obtain partial results
        # corresponding to all embedding lookups, but part of the batch on each device.
        # Therefore, matching the distribution of output of bottom mlp, so that both
        # could be used for subsequent interactions on each device.
        if len(self.emb_l) != len(ly):
            sys.exit("ERROR: corrupted intermediate result in parallel_forward call")

        t_list = []
        for k, _ in enumerate(self.emb_l):
            d = torch.device("cuda:" + str(k % ndevices))
            y = scatter(ly[k], device_ids, dim=0)
            t_list.append(y)
        # adjust the list to be ordered per device
        ly = list(map(lambda y: list(y), zip(*t_list)))
        # debug prints
        # print(ly)

        # interactions
        z = []
        for k in range(ndevices):
            zk = self.interact_features(x[k], ly[k])
            z.append(zk)
        # debug prints
        # print(z)

        # top mlp
        # WARNING: Note that the self.top_l is a list of top mlp modules that
        # have been replicated across devices, while z is a list of interaction results
        # that by construction are scattered across devices on the first (batch) dim.
        # The output is a list of tensors scattered across devices according to the
        # distribution of z.
        p = parallel_apply(self.top_l_replicas, z, None, device_ids)

        ### gather the distributed results ###
        p0 = gather(p, self.output_d, dim=0)

        # clamp output if needed
        if 0.0 < self.loss_threshold and self.loss_threshold < 1.0:
            z0 = torch.clamp(
                p0, min=self.loss_threshold, max=(1.0 - self.loss_threshold)
            )
        else:
            z0 = p0

        return z0

In [4]:
### parse arguments ###
# args_dict = dict(
#     activation_function='relu', arch_embedding_size='4-3-2', arch_interaction_itself=False, arch_interaction_op='dot', arch_mlp_bot='4-3-2', 
# arch_mlp_top='4-2-1', arch_sparse_feature_size=2, data_generation='random', data_randomize='total', 
# data_set='kaggle', data_size=1, data_sub_sample_rate=0.0, data_trace_enable_padding=False, 
# data_trace_file='./input/dist_emb_j.log', debug_mode=False, enable_profiling=False, inference_only=False, 
# learning_rate=0.01, load_model='', loss_function='mse', loss_threshold=0.0, loss_weights='1.0-1.0', 
# max_ind_range=-1, md_flag=False, md_round_dims=False, md_temperature=0.3, md_threshold=200, 
# memory_map=False, mini_batch_size=1, mlperf_acc_threshold=0.0, mlperf_auc_threshold=0.0, 
# mlperf_bin_loader=False, mlperf_bin_shuffle=False, mlperf_logging=False, nepochs=1, num_batches=0, 
# num_indices_per_lookup=10, num_indices_per_lookup_fixed=False, num_workers=0, numpy_rand_seed=123, 
# plot_compute_graph=False, print_freq=1, print_precision=5, print_time=False, processed_data_file='', 
# qr_collisions=4, qr_flag=False, qr_operation='mult', qr_threshold=200, raw_data_file='', 
# round_targets=False, save_model='', save_onnx=False, sync_dense_params=True, test_freq=-1, 
# test_mini_batch_size=-1, test_num_workers=-1, use_gpu=False      
# )


In [5]:
### parse arguments2 ###
args_dict = dict(
    activation_function='relu', arch_embedding_size='4-3-2', arch_interaction_itself=False, arch_interaction_op='dot', 
    arch_mlp_bot='13-512-256-64-16', arch_mlp_top='512-256-1', arch_sparse_feature_size=16, data_generation='dataset', 
    data_randomize='total', data_set='kaggle', data_size=1, data_sub_sample_rate=0.0, data_trace_enable_padding=False, 
    data_trace_file='./input/dist_emb_j.log', debug_mode=False, enable_profiling=False, inference_only=False, 
    learning_rate=0.1, load_model='', loss_function='bce', loss_threshold=0.0, loss_weights='1.0-1.0', 
    max_ind_range=-1, md_flag=False, md_round_dims=False, md_temperature=0.3, md_threshold=200, memory_map=False, 
    mini_batch_size=128, mlperf_acc_threshold=0.0, mlperf_auc_threshold=0.0, mlperf_bin_loader=False, 
    mlperf_bin_shuffle=False, mlperf_logging=False, nepochs=1, num_batches=0, num_indices_per_lookup=10, 
    num_indices_per_lookup_fixed=False, num_workers=0, numpy_rand_seed=123, plot_compute_graph=False, 
    print_freq=1024, print_precision=5, print_time=True, processed_data_file='', qr_collisions=4, 
    qr_flag=False, qr_operation='mult', qr_threshold=200, 
    raw_data_file='/Users/seuzha/Desktop/ml_data_set/criteo_lab/dac/train.txt', round_targets=True, 
    save_model='', save_onnx=False, sync_dense_params=True, test_freq=-1, test_mini_batch_size=16384, 
    test_num_workers=16, use_gpu=False     
)


In [6]:
args_dict 

{'activation_function': 'relu',
 'arch_embedding_size': '4-3-2',
 'arch_interaction_itself': False,
 'arch_interaction_op': 'dot',
 'arch_mlp_bot': '13-512-256-64-16',
 'arch_mlp_top': '512-256-1',
 'arch_sparse_feature_size': 16,
 'data_generation': 'dataset',
 'data_randomize': 'total',
 'data_set': 'kaggle',
 'data_size': 1,
 'data_sub_sample_rate': 0.0,
 'data_trace_enable_padding': False,
 'data_trace_file': './input/dist_emb_j.log',
 'debug_mode': False,
 'enable_profiling': False,
 'inference_only': False,
 'learning_rate': 0.1,
 'load_model': '',
 'loss_function': 'bce',
 'loss_threshold': 0.0,
 'loss_weights': '1.0-1.0',
 'max_ind_range': -1,
 'md_flag': False,
 'md_round_dims': False,
 'md_temperature': 0.3,
 'md_threshold': 200,
 'memory_map': False,
 'mini_batch_size': 128,
 'mlperf_acc_threshold': 0.0,
 'mlperf_auc_threshold': 0.0,
 'mlperf_bin_loader': False,
 'mlperf_bin_shuffle': False,
 'mlperf_logging': False,
 'nepochs': 1,
 'num_batches': 0,
 'num_indices_per_lookup

In [7]:
# a simple class example
class test:
    def __init__(self, input_dict):
        for key in input_dict:
            self.__dict__[key] = input_dict[key]
            
# benchmark
class test_benchmark:
    def __init__(self, a, b):
        self.a=a
        self.b=b


In [8]:
args = test(args_dict)

In [9]:
args.debug_mode

False

In [10]:
#change args as needed
args.debug_mode = True
args.test_freq = 1024
args.save_model = '/Users/seuzha/Desktop/ml_data_set/criteo_lab/dac/dummy_model.pt'

In [11]:
### some basic setup ###
np.random.seed(args.numpy_rand_seed)
np.set_printoptions(precision=args.print_precision)
torch.set_printoptions(precision=args.print_precision)
torch.manual_seed(args.numpy_rand_seed)

if (args.test_mini_batch_size < 0):
    # if the parameter is not set, use the training batch size
    args.test_mini_batch_size = args.mini_batch_size
if (args.test_num_workers < 0):
    # if the parameter is not set, use the same parameter for training
    args.test_num_workers = args.num_workers

use_gpu = args.use_gpu and torch.cuda.is_available()
if use_gpu:
    torch.cuda.manual_seed_all(args.numpy_rand_seed)
    torch.backends.cudnn.deterministic = True
    device = torch.device("cuda", 0)
    ngpus = torch.cuda.device_count()  # 1
    print("Using {} GPU(s)...".format(ngpus))
else:
    device = torch.device("cpu")
    print("Using CPU...")

### prepare training data ###
ln_bot = np.fromstring(args.arch_mlp_bot, dtype=int, sep="-")
# input data
if (args.data_generation == "dataset"):

    train_data, train_ld, test_data, test_ld = \
        dp.make_criteo_data_and_loaders(args)
    nbatches = args.num_batches if args.num_batches > 0 else len(train_ld)
    nbatches_test = len(test_ld)
    
    # a tensor of vocaburary counts for categorical features
    ln_emb = train_data.counts
    # enforce maximum limit on number of vectors per embedding
    if args.max_ind_range > 0:
        ln_emb = np.array(list(map(
            lambda x: x if x < args.max_ind_range else args.max_ind_range,
            ln_emb
        )))
    m_den = train_data.m_den
    ln_bot[0] = m_den
else:
    # input and target at random
    ln_emb = np.fromstring(args.arch_embedding_size, dtype=int, sep="-")
    m_den = ln_bot[0]
    train_data, train_ld = dp.make_random_data_and_loader(args, ln_emb, m_den)
    nbatches = args.num_batches if args.num_batches > 0 else len(train_ld)

Using CPU...
Reading raw data=/Users/seuzha/Desktop/ml_data_set/criteo_lab/dac/train.txt
Skipping counts per file (already exist)
Skip existing /Users/seuzha/Desktop/ml_data_set/criteo_lab/dac/train_day_0.npz
Skip existing /Users/seuzha/Desktop/ml_data_set/criteo_lab/dac/train_day_1.npz
Skip existing /Users/seuzha/Desktop/ml_data_set/criteo_lab/dac/train_day_2.npz
Skip existing /Users/seuzha/Desktop/ml_data_set/criteo_lab/dac/train_day_3.npz
Skip existing /Users/seuzha/Desktop/ml_data_set/criteo_lab/dac/train_day_4.npz
Skip existing /Users/seuzha/Desktop/ml_data_set/criteo_lab/dac/train_day_5.npz
Skip existing /Users/seuzha/Desktop/ml_data_set/criteo_lab/dac/train_day_6.npz
Total number of samples: 458409
Divided into days/splits:
 [65487, 65487, 65487, 65487, 65487, 65487, 65487]
Using existing /Users/seuzha/Desktop/ml_data_set/criteo_lab/dac/train_day_6_processed.npz
Concatenating multiple days into /Users/seuzha/Desktop/ml_data_set/criteo_lab/dac/kaggleAdDisplayChallenge_processed.n

In [12]:
#litez: the elements represents vocabulary size for each of the embedding element
ln_emb

array([   988,    542, 181190,  79476,    230,     14,  10417,    443,
            3,  23543,   4640, 163044,   3116,     26,   8074, 128285,
           10,   3611,   1692,      4, 148667,     15,     14,  29185,
           65,  21828], dtype=int32)

In [13]:
m_den

13

In [14]:
train_data

In [15]:
train_ld.dataset

In [16]:
args.arch_sparse_feature_size

16

In [17]:
ln_emb.size

26

In [18]:
ln_bot

array([ 13, 512, 256,  64,  16])

In [19]:
nbatches

3070

In [20]:
len(train_ld)

3070

In [21]:
len(test_ld)

2

In [22]:
3070*128

392960

In [23]:
args.arch_sparse_feature_size

16

In [24]:
### parse command line arguments ###
#litez: essentially this is the embedding_dim;
# may want a better name fot it
m_spa = args.arch_sparse_feature_size

# litez: here number of dense features is 1
num_fea = ln_emb.size + 1  # num sparse + num dense features

# litez: do we want to make sure that  m_spa=m_den_out?
# otherwise, there may be problems when we are doing feature interaction
m_den_out = ln_bot[ln_bot.size - 1]
if args.arch_interaction_op == "dot":
    # approach 1: all
    # num_int = num_fea * num_fea + m_den_out
    # approach 2: unique
    if args.arch_interaction_itself:
        num_int = (num_fea * (num_fea + 1)) // 2 + m_den_out
    else:
        num_int = (num_fea * (num_fea - 1)) // 2 + m_den_out
elif args.arch_interaction_op == "cat":
    num_int = num_fea * m_den_out
else:
    sys.exit(
        "ERROR: --arch-interaction-op="
        + args.arch_interaction_op
        + " is not supported"
    )

# litez: actually, it is kind of conter-inuitive to adjust the top-layer number
# as the definition is not consistent to bottom mlp; we can probably specify 
# str(num_ini) as an input; of course then, we need to be careful on choosing `dot`
# or `cat` operator for feature interactions. One related question is `cat` ever used 
# in practice?
arch_mlp_top_adjusted = str(num_int) + "-" + args.arch_mlp_top
ln_top = np.fromstring(arch_mlp_top_adjusted, dtype=int, sep="-")

# sanity check: feature sizes and mlp dimensions must match
if m_den != ln_bot[0]:
    sys.exit(
        "ERROR: arch-dense-feature-size "
        + str(m_den)
        + " does not match first dim of bottom mlp "
        + str(ln_bot[0])
    )
if args.qr_flag:
    if args.qr_operation == "concat" and 2 * m_spa != m_den_out:
        sys.exit(
            "ERROR: 2 arch-sparse-feature-size "
            + str(2 * m_spa)
            + " does not match last dim of bottom mlp "
            + str(m_den_out)
            + " (note that the last dim of bottom mlp must be 2x the embedding dim)"
        )
    if args.qr_operation != "concat" and m_spa != m_den_out:
        sys.exit(
            "ERROR: arch-sparse-feature-size "
            + str(m_spa)
            + " does not match last dim of bottom mlp "
            + str(m_den_out)
        )
else:
    if m_spa != m_den_out:
        sys.exit(
            "ERROR: arch-sparse-feature-size "
            + str(m_spa)
            + " does not match last dim of bottom mlp "
            + str(m_den_out)
        )
if num_int != ln_top[0]:
    sys.exit(
        "ERROR: # of feature interactions "
        + str(num_int)
        + " does not match first dimension of top mlp "
        + str(ln_top[0])
    )

# assign mixed dimensions if applicable
if args.md_flag:
    m_spa = md_solver(
        torch.tensor(ln_emb),
        args.md_temperature,  # alpha
        d0=m_spa,
        round_dim=args.md_round_dims
    ).tolist()

# test prints (model arch)
if args.debug_mode:
    print("model arch:")
    print(
        "mlp top arch "
        + str(ln_top.size - 1)
        + " layers, with input to output dimensions:"
    )
    print(ln_top)
    print("# of interactions")
    print(num_int)
    print(
        "mlp bot arch "
        + str(ln_bot.size - 1)
        + " layers, with input to output dimensions:"
    )
    print(ln_bot)
    print("# of features (sparse and dense)")
    print(num_fea)
    print("dense feature size")
    print(m_den)
    print("sparse feature size")
    print(m_spa)
    print(
        "# of embeddings (= # of sparse features) "
        + str(ln_emb.size)
        + ", with dimensions "
        + str(m_spa)
        + "x:"
    )
    print(ln_emb)

    print("data (inputs and targets):")
    for j, (X, lS_o, lS_i, T) in enumerate(train_ld):
        # early exit if nbatches was set by the user and has been exceeded
        if nbatches > 0 and j >= 2:
            break

        print("mini-batch: %d" % j)
        print(X.detach().cpu().numpy())
        # transform offsets to lengths when printing
        print(
            [
                np.diff(
                    S_o.detach().cpu().tolist() + list(lS_i[i].shape)
                ).tolist()
                for i, S_o in enumerate(lS_o)
            ]
        )
        print([S_i.detach().cpu().tolist() for S_i in lS_i])
        print(T.detach().cpu().numpy())

ndevices = min(ngpus, args.mini_batch_size, num_fea - 1) if use_gpu else -1

model arch:
mlp top arch 3 layers, with input to output dimensions:
[367 512 256   1]
# of interactions
367
mlp bot arch 4 layers, with input to output dimensions:
[ 13 512 256  64  16]
# of features (sparse and dense)
27
dense feature size
13
sparse feature size
16
# of embeddings (= # of sparse features) 26, with dimensions 16x:
[   988    542 181190  79476    230     14  10417    443      3  23543
   4640 163044   3116     26   8074 128285     10   3611   1692      4
 148667     15     14  29185     65  21828]
data (inputs and targets):
mini-batch: 0
[[1.94591 4.63473 1.79176 ... 2.56495 4.07754 1.09861]
 [0.      0.      1.94591 ... 0.      0.      1.09861]
 [0.      5.69709 1.09861 ... 0.69315 0.      1.38629]
 ...
 [0.      4.58497 1.09861 ... 0.      0.      0.69315]
 [0.69315 0.69315 1.09861 ... 0.69315 0.      0.     ]
 [0.      0.      0.      ... 0.69315 0.      0.69315]]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [25]:
# for k, sparse_index_group_batch in enumerate(lS_i):

In [26]:
# for j, (X, lS_o, lS_i, T) in enumerate(train_ld):
#     if j==0:
#         print(X.detach().cpu().numpy().shape)
#         print('len lS_o', len(lS_o))
#         for idx, S_o in enumerate(lS_o):
#             print('len of single S_o', S_o.shape)
#             if idx ==0:
#                 print('example single S_o', S_o)
#         print('len lS_i', len(lS_i))
#         for idx, S_i in enumerate(lS_i):
#             print('len of signle S_i', S_i.shape)
#             if idx ==0:
#                 print('example single S_i', S_i)
#         print('len T', len(T))

# print(j)

In [27]:
# litez: train_data looks wired...
train_data[1]

(array([ 0,  0,  6,  2, 14,  2, 15,  2,  2,  0,  0,  0,  2], dtype=int32),
 array([0.000e+00, 3.600e+01, 5.745e+03, 4.056e+03, 0.000e+00, 0.000e+00,
        9.700e+01, 1.000e+00, 0.000e+00, 4.770e+02, 9.000e+01, 5.249e+03,
        8.700e+01, 1.000e+00, 5.400e+01, 4.702e+03, 0.000e+00, 4.500e+01,
        0.000e+00, 3.000e+00, 4.994e+03, 1.000e+00, 7.000e+00, 1.200e+01,
        1.100e+01, 2.000e+01]),
 1)

In [28]:


### construct the neural network specified above ###
# WARNING: to obtain exactly the same initialization for
# the weights we need to start from the same random seed.
# np.random.seed(args.numpy_rand_seed)
dlrm = DLRM_Net(
    m_spa,
    ln_emb,
    ln_bot,
    ln_top,
    arch_interaction_op=args.arch_interaction_op,
    arch_interaction_itself=args.arch_interaction_itself,
    sigmoid_bot=-1,
    sigmoid_top=ln_top.size - 2,
    sync_dense_params=args.sync_dense_params,
    loss_threshold=args.loss_threshold,
    ndevices=ndevices,
    qr_flag=args.qr_flag,
    qr_operation=args.qr_operation,
    qr_collisions=args.qr_collisions,
    qr_threshold=args.qr_threshold,
    md_flag=args.md_flag,
    md_threshold=args.md_threshold,
)
# test prints
if args.debug_mode:
    print("initial parameters (weights and bias):")
    for param in dlrm.parameters():
        print(param.detach().cpu().numpy())
    # print(dlrm)

if use_gpu:
    # Custom Model-Data Parallel
    # the mlps are replicated and use data parallelism, while
    # the embeddings are distributed and use model parallelism
    dlrm = dlrm.to(device)  # .cuda()
    if dlrm.ndevices > 1:
        dlrm.emb_l = dlrm.create_emb(m_spa, ln_emb)

# specify the loss function
if args.loss_function == "mse":
    loss_fn = torch.nn.MSELoss(reduction="mean")
elif args.loss_function == "bce":
    loss_fn = torch.nn.BCELoss(reduction="mean")
elif args.loss_function == "wbce":
    loss_ws = torch.tensor(np.fromstring(args.loss_weights, dtype=float, sep="-"))
    loss_fn = torch.nn.BCELoss(reduction="none")
else:
    sys.exit("ERROR: --loss-function=" + args.loss_function + " is not supported")

if not args.inference_only:
    # specify the optimizer algorithm
    optimizer = torch.optim.SGD(dlrm.parameters(), lr=args.learning_rate)

### main loop ###
def time_wrap(use_gpu):
    if use_gpu:
        torch.cuda.synchronize()
    return time.time()

def dlrm_wrap(X, lS_o, lS_i, use_gpu, device):
    if use_gpu:  # .cuda()
        # lS_i can be either a list of tensors or a stacked tensor.
        # Handle each case below:
        lS_i = [S_i.to(device) for S_i in lS_i] if isinstance(lS_i, list) \
            else lS_i.to(device)
        lS_o = [S_o.to(device) for S_o in lS_o] if isinstance(lS_o, list) \
            else lS_o.to(device)
        return dlrm(
            X.to(device),
            lS_o,
            lS_i
        )
    else:
        return dlrm(X, lS_o, lS_i)

def loss_fn_wrap(Z, T, use_gpu, device):
    if args.loss_function == "mse" or args.loss_function == "bce":
        if use_gpu:
            return loss_fn(Z, T.to(device))
        else:
            return loss_fn(Z, T)
    elif args.loss_function == "wbce":
        if use_gpu:
            loss_ws_ = loss_ws[T.data.view(-1).long()].view_as(T).to(device)
            loss_fn_ = loss_fn(Z, T.to(device))
        else:
            loss_ws_ = loss_ws[T.data.view(-1).long()].view_as(T)
            loss_fn_ = loss_fn(Z, T.to(device))
        loss_sc_ = loss_ws_ * loss_fn_
        # debug prints
        # print(loss_ws_)
        # print(loss_fn_)
        return loss_sc_.mean()

# training or inference
best_gA_test = 0
best_auc_test = 0
skip_upto_epoch = 0
skip_upto_batch = 0
total_time = 0
total_loss = 0
total_accu = 0
total_iter = 0
total_samp = 0
k = 0

# Load model is specified
if not (args.load_model == ""):
    print("Loading saved model {}".format(args.load_model))
    if use_gpu:
        if dlrm.ndevices > 1:
            # NOTE: when targeting inference on multiple GPUs,
            # load the model as is on CPU or GPU, with the move
            # to multiple GPUs to be done in parallel_forward
            ld_model = torch.load(args.load_model)
        else:
            # NOTE: when targeting inference on single GPU,
            # note that the call to .to(device) has already happened
            ld_model = torch.load(
                args.load_model,
                map_location=torch.device('cuda')
                # map_location=lambda storage, loc: storage.cuda(0)
            )
    else:
        # when targeting inference on CPU
        ld_model = torch.load(args.load_model, map_location=torch.device('cpu'))
    dlrm.load_state_dict(ld_model["state_dict"])
    ld_j = ld_model["iter"]
    ld_k = ld_model["epoch"]
    ld_nepochs = ld_model["nepochs"]
    ld_nbatches = ld_model["nbatches"]
    ld_nbatches_test = ld_model["nbatches_test"]
    ld_gA = ld_model["train_acc"]
    ld_gL = ld_model["train_loss"]
    ld_total_loss = ld_model["total_loss"]
    ld_total_accu = ld_model["total_accu"]
    ld_gA_test = ld_model["test_acc"]
    ld_gL_test = ld_model["test_loss"]
    if not args.inference_only:
        optimizer.load_state_dict(ld_model["opt_state_dict"])
        best_gA_test = ld_gA_test
        total_loss = ld_total_loss
        total_accu = ld_total_accu
        skip_upto_epoch = ld_k  # epochs
        skip_upto_batch = ld_j  # batches
    else:
        args.print_freq = ld_nbatches
        args.test_freq = 0

    print(
        "Saved at: epoch = {:d}/{:d}, batch = {:d}/{:d}, ntbatch = {:d}".format(
            ld_k, ld_nepochs, ld_j, ld_nbatches, ld_nbatches_test
        )
    )
    print(
        "Training state: loss = {:.6f}, accuracy = {:3.3f} %".format(
            ld_gL, ld_gA * 100
        )
    )
    print(
        "Testing state: loss = {:.6f}, accuracy = {:3.3f} %".format(
            ld_gL_test, ld_gA_test * 100
        )
    )

initial parameters (weights and bias):
[[ 0.0151   0.03055 -0.02602 ...  0.00402 -0.02836  0.00095]
 [-0.0047   0.02269 -0.03016 ...  0.01263 -0.00026 -0.00398]
 [ 0.02784 -0.01681  0.00642 ... -0.00195  0.02449  0.01874]
 ...
 [-0.011    0.01685  0.00188 ...  0.00729 -0.00099  0.02803]
 [-0.00966  0.02676 -0.01924 ...  0.02036  0.02827  0.0112 ]
 [ 0.00147 -0.00756  0.00132 ... -0.02577  0.00206  0.02213]]
[[-0.03911  0.01796  0.01742 ... -0.00433  0.02673 -0.01448]
 [ 0.00036 -0.01477  0.03319 ... -0.03843  0.01802  0.00049]
 [-0.0102  -0.01917  0.03319 ...  0.002   -0.02244 -0.00371]
 ...
 [ 0.00834  0.01661 -0.0043  ... -0.00746  0.00449  0.0363 ]
 [ 0.01619 -0.03124  0.0121  ...  0.02685  0.04001  0.03847]
 [-0.03661 -0.03387 -0.01511 ...  0.01051 -0.0212  -0.04137]]
[[ 2.47746e-05  3.27817e-04 -8.39343e-04 ...  1.02215e-04 -2.27285e-03
  -1.74064e-03]
 [ 2.00169e-04 -2.11335e-03  6.27219e-04 ...  1.01414e-03  5.08524e-04
  -7.84518e-04]
 [ 9.34161e-04  2.02637e-03 -1.25288e-03 ..

In [29]:
dlrm

DLRM_Net(
  (emb_l): ModuleList(
    (0): EmbeddingBag(988, 16, mode=sum)
    (1): EmbeddingBag(542, 16, mode=sum)
    (2): EmbeddingBag(181190, 16, mode=sum)
    (3): EmbeddingBag(79476, 16, mode=sum)
    (4): EmbeddingBag(230, 16, mode=sum)
    (5): EmbeddingBag(14, 16, mode=sum)
    (6): EmbeddingBag(10417, 16, mode=sum)
    (7): EmbeddingBag(443, 16, mode=sum)
    (8): EmbeddingBag(3, 16, mode=sum)
    (9): EmbeddingBag(23543, 16, mode=sum)
    (10): EmbeddingBag(4640, 16, mode=sum)
    (11): EmbeddingBag(163044, 16, mode=sum)
    (12): EmbeddingBag(3116, 16, mode=sum)
    (13): EmbeddingBag(26, 16, mode=sum)
    (14): EmbeddingBag(8074, 16, mode=sum)
    (15): EmbeddingBag(128285, 16, mode=sum)
    (16): EmbeddingBag(10, 16, mode=sum)
    (17): EmbeddingBag(3611, 16, mode=sum)
    (18): EmbeddingBag(1692, 16, mode=sum)
    (19): EmbeddingBag(4, 16, mode=sum)
    (20): EmbeddingBag(148667, 16, mode=sum)
    (21): EmbeddingBag(15, 16, mode=sum)
    (22): EmbeddingBag(14, 16, mode=su

In [30]:
skip_upto_epoch

0

In [31]:
args.nepochs

1

In [32]:
print("time/loss/accuracy (if enabled):")
with torch.autograd.profiler.profile(args.enable_profiling, use_gpu) as prof:
    while k < args.nepochs:
        if k < skip_upto_epoch:
            continue

        accum_time_begin = time_wrap(use_gpu)

        if args.mlperf_logging:
            previous_iteration_time = None

        for j, (X, lS_o, lS_i, T) in enumerate(train_ld):
            if j < skip_upto_batch:
                continue

            if args.mlperf_logging:
                current_time = time_wrap(use_gpu)
                if previous_iteration_time:
                    iteration_time = current_time - previous_iteration_time
                else:
                    iteration_time = 0
                previous_iteration_time = current_time
            else:
                t1 = time_wrap(use_gpu)

            # early exit if nbatches was set by the user and has been exceeded
            if nbatches > 0 and j >= nbatches:
                break
            '''
            # debug prints
            print("input and targets")
            print(X.detach().cpu().numpy())
            print([np.diff(S_o.detach().cpu().tolist()
                   + list(lS_i[i].shape)).tolist() for i, S_o in enumerate(lS_o)])
            print([S_i.detach().cpu().numpy().tolist() for S_i in lS_i])
            print(T.detach().cpu().numpy())
            '''

            # forward pass
            Z = dlrm_wrap(X, lS_o, lS_i, use_gpu, device)

            # loss
            E = loss_fn_wrap(Z, T, use_gpu, device)
            '''
            # debug prints
            print("output and loss")
            print(Z.detach().cpu().numpy())
            print(E.detach().cpu().numpy())
            '''
            # compute loss and accuracy
            L = E.detach().cpu().numpy()  # numpy array
            S = Z.detach().cpu().numpy()  # numpy array
            T = T.detach().cpu().numpy()  # numpy array
            mbs = T.shape[0]  # = args.mini_batch_size except maybe for last
            A = np.sum((np.round(S, 0) == T).astype(np.uint8))

            if not args.inference_only:
                # scaled error gradient propagation
                # (where we do not accumulate gradients across mini-batches)
                optimizer.zero_grad()
                # backward pass
                E.backward()
                # debug prints (check gradient norm)
                # for l in mlp.layers:
                #     if hasattr(l, 'weight'):
                #          print(l.weight.grad.norm().item())

                # optimizer
                optimizer.step()

            if args.mlperf_logging:
                total_time += iteration_time
            else:
                t2 = time_wrap(use_gpu)
                total_time += t2 - t1
            total_accu += A
            total_loss += L * mbs
            total_iter += 1
            total_samp += mbs

            should_print = ((j + 1) % args.print_freq == 0) or (j + 1 == nbatches)
            should_test = (
                (args.test_freq > 0)
                and (args.data_generation == "dataset")
                and (((j + 1) % args.test_freq == 0) or (j + 1 == nbatches))
            )

            # print time, loss and accuracy
            if should_print or should_test:
                gT = 1000.0 * total_time / total_iter if args.print_time else -1
                total_time = 0

                gA = total_accu / total_samp
                total_accu = 0

                gL = total_loss / total_samp
                total_loss = 0

                str_run_type = "inference" if args.inference_only else "training"
                print(
                    "Finished {} it {}/{} of epoch {}, {:.2f} ms/it, ".format(
                        str_run_type, j + 1, nbatches, k, gT
                    )
                    + "loss {:.6f}, accuracy {:3.3f} %".format(gL, gA * 100)
                )
                # Uncomment the line below to print out the total time with overhead
                # print("Accumulated time so far: {}" \
                # .format(time_wrap(use_gpu) - accum_time_begin))
                total_iter = 0
                total_samp = 0

            # testing
            if should_test and not args.inference_only:
                # don't measure training iter time in a test iteration
                if args.mlperf_logging:
                    previous_iteration_time = None

                test_accu = 0
                test_loss = 0
                test_samp = 0

                accum_test_time_begin = time_wrap(use_gpu)
                if args.mlperf_logging:
                    scores = []
                    targets = []

                for i, (X_test, lS_o_test, lS_i_test, T_test) in enumerate(test_ld):
                    # early exit if nbatches was set by the user and was exceeded
                    if nbatches > 0 and i >= nbatches:
                        break

                    t1_test = time_wrap(use_gpu)

                    # forward pass
                    Z_test = dlrm_wrap(
                        X_test, lS_o_test, lS_i_test, use_gpu, device
                    )
                    if args.mlperf_logging:
                        S_test = Z_test.detach().cpu().numpy()  # numpy array
                        T_test = T_test.detach().cpu().numpy()  # numpy array
                        scores.append(S_test)
                        targets.append(T_test)
                    else:
                        # loss
                        E_test = loss_fn_wrap(Z_test, T_test, use_gpu, device)

                        # compute loss and accuracy
                        L_test = E_test.detach().cpu().numpy()  # numpy array
                        S_test = Z_test.detach().cpu().numpy()  # numpy array
                        T_test = T_test.detach().cpu().numpy()  # numpy array
                        mbs_test = T_test.shape[0]  # = mini_batch_size except last
                        A_test = np.sum((np.round(S_test, 0) == T_test).astype(np.uint8))
                        test_accu += A_test
                        test_loss += L_test * mbs_test
                        test_samp += mbs_test

                    t2_test = time_wrap(use_gpu)

                if args.mlperf_logging:
                    scores = np.concatenate(scores, axis=0)
                    targets = np.concatenate(targets, axis=0)

                    metrics = {
                        'loss' : sklearn.metrics.log_loss,
                        'recall' : lambda y_true, y_score:
                        sklearn.metrics.recall_score(
                            y_true=y_true,
                            y_pred=np.round(y_score)
                        ),
                        'precision' : lambda y_true, y_score:
                        sklearn.metrics.precision_score(
                            y_true=y_true,
                            y_pred=np.round(y_score)
                        ),
                        'f1' : lambda y_true, y_score:
                        sklearn.metrics.f1_score(
                            y_true=y_true,
                            y_pred=np.round(y_score)
                        ),
                        'ap' : sklearn.metrics.average_precision_score,
                        'roc_auc' : sklearn.metrics.roc_auc_score,
                        'accuracy' : lambda y_true, y_score:
                        sklearn.metrics.accuracy_score(
                            y_true=y_true,
                            y_pred=np.round(y_score)
                        ),
                        # 'pre_curve' : sklearn.metrics.precision_recall_curve,
                        # 'roc_curve' :  sklearn.metrics.roc_curve,
                    }

                    # print("Compute time for validation metric : ", end="")
                    # first_it = True
                    validation_results = {}
                    for metric_name, metric_function in metrics.items():
                        # if first_it:
                        #     first_it = False
                        # else:
                        #     print(", ", end="")
                        # metric_compute_start = time_wrap(False)
                        validation_results[metric_name] = metric_function(
                            targets,
                            scores
                        )
                        # metric_compute_end = time_wrap(False)
                        # met_time = metric_compute_end - metric_compute_start
                        # print("{} {:.4f}".format(metric_name, 1000 * (met_time)),
                        #      end="")
                    # print(" ms")
                    gA_test = validation_results['accuracy']
                    gL_test = validation_results['loss']
                else:
                    gA_test = test_accu / test_samp
                    gL_test = test_loss / test_samp

                is_best = gA_test > best_gA_test
                if is_best:
                    best_gA_test = gA_test
                    if not (args.save_model == ""):
                        print("Saving model to {}".format(args.save_model))
                        torch.save(
                            {
                                "epoch": k,
                                "nepochs": args.nepochs,
                                "nbatches": nbatches,
                                "nbatches_test": nbatches_test,
                                "iter": j + 1,
                                "state_dict": dlrm.state_dict(),
                                "train_acc": gA,
                                "train_loss": gL,
                                "test_acc": gA_test,
                                "test_loss": gL_test,
                                "total_loss": total_loss,
                                "total_accu": total_accu,
                                "opt_state_dict": optimizer.state_dict(),
                            },
                            args.save_model,
                        )

                if args.mlperf_logging:
                    is_best = validation_results['roc_auc'] > best_auc_test
                    if is_best:
                        best_auc_test = validation_results['roc_auc']

                    print(
                        "Testing at - {}/{} of epoch {},".format(j + 1, nbatches, k)
                        + " loss {:.6f}, recall {:.4f}, precision {:.4f},".format(
                            validation_results['loss'],
                            validation_results['recall'],
                            validation_results['precision']
                        )
                        + " f1 {:.4f}, ap {:.4f},".format(
                            validation_results['f1'],
                            validation_results['ap'],
                        )
                        + " auc {:.4f}, best auc {:.4f},".format(
                            validation_results['roc_auc'],
                            best_auc_test
                        )
                        + " accuracy {:3.3f} %, best accuracy {:3.3f} %".format(
                            validation_results['accuracy'] * 100,
                            best_gA_test * 100
                        )
                    )
                else:
                    print(
                        "Testing at - {}/{} of epoch {},".format(j + 1, nbatches, 0)
                        + " loss {:.6f}, accuracy {:3.3f} %, best {:3.3f} %".format(
                            gL_test, gA_test * 100, best_gA_test * 100
                        )
                    )
                # Uncomment the line below to print out the total time with overhead
                # print("Total test time for this group: {}" \
                # .format(time_wrap(use_gpu) - accum_test_time_begin))

                if (args.mlperf_logging
                    and (args.mlperf_acc_threshold > 0)
                    and (best_gA_test > args.mlperf_acc_threshold)):
                    print("MLPerf testing accuracy threshold "
                          + str(args.mlperf_acc_threshold)
                          + " reached, stop training")
                    break

                if (args.mlperf_logging
                    and (args.mlperf_auc_threshold > 0)
                    and (best_auc_test > args.mlperf_auc_threshold)):
                    print("MLPerf testing auc threshold "
                          + str(args.mlperf_auc_threshold)
                          + " reached, stop training")
                    break

        k += 1  # nepochs

time/loss/accuracy (if enabled):
Finished training it 1024/3070 of epoch 0, 24.21 ms/it, loss 0.519843, accuracy 75.326 %
Saving model to /Users/seuzha/Desktop/ml_data_set/criteo_lab/dac/dummy_model.pt
Testing at - 1024/3070 of epoch 0, loss 0.515687, accuracy 75.489 %, best 75.489 %
Finished training it 2048/3070 of epoch 0, 24.60 ms/it, loss 0.501983, accuracy 76.567 %
Testing at - 2048/3070 of epoch 0, loss 0.513784, accuracy 75.351 %, best 75.489 %
Finished training it 3070/3070 of epoch 0, 25.28 ms/it, loss 0.497440, accuracy 76.692 %
Saving model to /Users/seuzha/Desktop/ml_data_set/criteo_lab/dac/dummy_model.pt
Testing at - 3070/3070 of epoch 0, loss 0.508248, accuracy 76.051 %, best 76.051 %
